In [1]:
import os
import datetime
from datetime import timedelta
import numpy as np
from funcs import get_parc_sizes, extract
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib

In [2]:
def timeTicks(y, pos):                                                                                                                                                                                                                                                         
    d = datetime.timedelta(seconds=y)                                                                                                                                                                                                                                          
    return str(d)

def set_y_time(ax):
    formatter = matplotlib.ticker.FuncFormatter(timeTicks)                                                                                                                                                                                                                         
    ax.yaxis.set_major_formatter(formatter)
    
def round_size(df, r):
    df['size'] = (df['size'] * 1/r).astype(int) * r
    return df

def proc_file(file, dr):

    full_path = os.path.join(dr, file)

    # Open file
    with open(full_path, 'r') as f:
        txt = f.readlines()

    # Check for multiple runs
    is_new_run = ['run  ' in line for line in txt]

    # Multiple run case
    if sum(is_new_run) >= 2:
        inds = list(np.where(is_new_run)[0]) + [None]

        for i in range(len(inds)-1):
            add(txt[inds[i]:inds[i+1]])

    # Single run case
    else:
        add(txt)
        
def proc_dr(dr):

    files = os.listdir(dr)
    files = [file for file in files if '.out' in file]

    for file in files:
        proc_file(file, dr)

In [3]:
# Load parc sizes
parc_sizes = get_parc_sizes(everything=True)

# Init timing df
timing = pd.DataFrame(columns=['model', 'size', 'is_b',
                               'time', 'p_type',
                               'n_jobs', 'n_load_saved'])

# Helper function
def add(file_txt):
    ex = extract(file_txt, parc_sizes)
    if ex is not None:
        timing.loc[len(timing)] = list(ex)


# Special case
proc_dr('../exp/All_Job_Logs/Original_Job_Logs_Ignore_SVM')
timing.drop(timing[timing['model'] == 'svm'].index, inplace=True)

# Process rest, Job_Logs1-9
for i in range(1, 10):
    print(i)
    proc_dr(f'../exp/All_Job_Logs/Job_Logs{i}')

1
2
3
4
5
6


ValueError: True is not in list

In [ ]:
# Gen copy with no load
no_load = timing.loc[timing[timing['n_load_saved'] == 0].index]

In [ ]:
timing.shape

In [ ]:
no_load.shape

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

sns.scatterplot(data=no_load, alpha=.2,
                x="size", y="time", ax=ax,
                hue="model", marker="x", s=50)
set_y_time(ax)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
sns.violinplot(data=round_size(no_load, 500), cut=0,
               x="size", y="time", ax=ax)
set_y_time(ax)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

sns.violinplot(data=round_size(no_load, 500),
               cut=0,
               x="size",
               y="time",
               ax=ax,
               hue="model")

set_y_time(ax)